In [1]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

  Using cached ibm_db-3.1.0-cp37-cp37m-linux_x86_64.whl
  Using cached ibm_db_sa-0.3.3-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 43.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 43.6 MB/s eta 0:00:00
  Attempting uninstall: ibm_db
    Found existing installation: ibm-db 3.1.0
    Uninstalling ibm-db-3.1.0:
      Successfully uninstalled ibm-db-3.1.0
  Attempting uninstall: zipp
    Found existing installation: zipp 3.11.0
    Uninstalling zipp-3.11.0:
      Successfully uninstalled zipp-3.11.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.4
    Uninstalling importlib-metadata-4.11.4:
      Successfully uninstalled importlib-metadata-4.11.4
  Attempting uninstall: sqlalche

In [2]:
%load_ext sql

In [3]:
!pip install -q pandas==1.1.5

In [4]:
import csv, sqlite3

con = sqlite3.connect("my_data.db")
cur = con.cursor()

In [5]:
import pandas
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


In [6]:
%sql sqlite:///my_data.db

'Connected: @my_data.db'

## Problems

Now write and execute SQL queries to solve assignment problems

### Problem 1

##### Find the total number of crimes recorded in the CRIME table.


In [7]:
%sql select count(*) from chicago_crime_data; 

 * sqlite:///my_data.db
Done.


count(*)
533


### Problem 2

##### List community areas with per capita income less than 11000.


In [8]:
%sql select community_area_number, community_area_name from census_data where per_capita_income < 11000

 * sqlite:///my_data.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
26.0,West Garfield Park
30.0,South Lawndale
37.0,Fuller Park
54.0,Riverdale


### Problem 3

##### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)


In [10]:
%%sql 
select id,case_number, date, description
from chicago_crime_data 
where  DESCRIPTION like '%MINOR%'

 * sqlite:///my_data.db
Done.


ID,CASE_NUMBER,DATE,DESCRIPTION
3987219,HL266884,2005-03-31,SELL/GIVE/DEL LIQUOR TO MINOR
3266814,HK238408,2004-03-13,ILLEGAL CONSUMPTION BY MINOR


### Problem 4

##### List all kidnapping crimes involving a child?


In [14]:
%%sql 
select id,case_number, date,primary_type , description
from chicago_crime_data 
where primary_type like '%KIDNAPPING%' 
and description like '%CHILD%';

 * sqlite:///my_data.db
Done.


ID,CASE_NUMBER,DATE,PRIMARY_TYPE,DESCRIPTION
5276766,HN144152,2007-01-26,KIDNAPPING,CHILD ABDUCTION/STRANGER


### Problem 5

##### What kinds of crimes were recorded at schools?


In [22]:
%%sql 
select distinct primary_type 
from CHICAGO_CRIME_DATA 
where location_description like '%school%'

 * sqlite:///my_data.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6

##### List the average safety score for each type of school.


In [28]:
%%sql
select "Elementary, Middle, or High School", avg(safety_score) as score_rate
from chicago_public_schools_data
group by "Elementary, Middle, or High School"

 * sqlite:///my_data.db
Done.


"Elementary, Middle, or High School",score_rate
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7

##### List 5 community areas with highest % of households below poverty line


In [31]:
%%sql 
select community_area_name, percent_households_below_poverty 
from census_data 
order by percent_households_below_poverty desc
limit 3

 * sqlite:///my_data.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6


### Problem 8

##### Which community area is most crime prone?


In [39]:
%%sql 
select community_area_number, count(*) as number_of_crime
from chicago_crime_data
group by community_area_number
order by number_of_crime desc 
limit 1

 * sqlite:///my_data.db
Done.


COMMUNITY_AREA_NUMBER,number_of_crime
25.0,43


Double-click **here** for a hint

<!--
Query for the 'community area number' that is most crime prone.
-->


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index


In [40]:
%%sql  
select community_area_name 
from  ( select community_area_name , max(hardship_index) from census_data  )

 * sqlite:///my_data.db
Done.


community_area_name
Riverdale


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [41]:
%%sql  
select  cd.community_area_number, community_area_name  , cases
from census_data cd,    (select community_area_number , cases
                               from 
                                  (select community_area_number, count(case_number) cases
                                   from CHICAGO_CRIME_DATA
                                   group by community_area_number
                                   order by count(case_number) desc
                                   limit 1
                                  )       ) cc
where cd.community_area_number=cc.community_area_number

 * sqlite:///my_data.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,cases
25.0,Austin,43
